# Build a Convolutional Neural Network to classify images

In this notebook we show how to build a Convolutiona Neural Network, a foundational component in image processing, to classify images on the CIFAR dataset. It is just a simple tutorial in Tensorflow v2.

## Step 1: Installing and loading the libraries

In [ ]:
import tensorflow as tf

In [ ]:
# Check tensorflow version
print(tf.__version__)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix

from  tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split

%matplotlib inline

## Step 2: Data preprocessing

Load the CIFAR10 datasets from keras library

In [ ]:
# Load the dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# Extract a validation dataset
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In order to show the results using class names instead of number, we set an array with the target labels

In [ ]:
class_names = ['airplane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

As usual in image processing we have to normalize the images

In [ ]:
# Normalize train and test dataset
X_train = X_train / 255.
X_val = X_val / 255.
X_test = X_test / 255.
# Print the shape of the datasets
print('Train dataset shape: ', X_train.shape)
print('Validation dataset shape: ', X_val.shape)
print('Test dataset shape: ', X_test.shape)


In [ ]:
X_train[0]

Lets print an image

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i])
    # The CIFAR labels happen to be arrays,
    # which is why you need the extra index
    plt.xlabel(class_names[y_train[i][0]])
plt.show()

## Step 2: Build the CNN

Our CNN will take as inputs tensors with shape (image height, image width, channels) where channels refers to (R,G,B) image format. In this example the shape will be (32, 32, 3), so we have to set the input shape of our first layer.



In [ ]:
# Define a Sequential model
model = tf.keras.models.Sequential()

### Add the first convolutional layer

Hyperparameters of the the first layer:

- Filters: 32
- Kernel size: 3
- padding: same
- Activation function: relu
- input_shape: (32, 32, 3)

In [ ]:
# Add a convolutional layer
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[32, 32, 3]))

### Add a second Convolutional layer and a Max Poolling layer

Hyperparameters of the second Conv layer (same as before):
- Filters: 32
- Kernel size: 3
- padding: same
- Activation function: relu

But now we do not need to set the input shape, it is infered from the previous layer

Hyperparameters of the MaxPool layer:
- pool_size: 2
- strides: 2
- padding: valid

In [ ]:
# Add a convolutional layer
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
# Add a MAxPooling layer
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

### Add the thrid convolutional layer

The count of filters in this layer will be 64, we repeat the same parameters as before.



In [ ]:
# Add a convolutional layer
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))

### Add a final Conv2D and MaxPool layer

In [ ]:
# Add a convolutional layer
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
# Add a Pooling layer
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

### Add a flatenning layer

The output from the Convolutional layers acts as the input to the feed forward layer, we need to flatten the that output

In [ ]:
model.add(tf.keras.layers.Flatten())

### Add a Fully connected network

The clasification layer will be a fully connected network with two layers. First layer will have 128 units and the second one 10 units, the number of classes to predict. We want the probability of every class label as the output, then we use a softmax activation function


In [ ]:
# Add the first dense layer
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
# Add the final dense layer
model.add(tf.keras.layers.Dense(units=10, activation='relu'))


Show the summary of our nn

In [ ]:
model.summary()

## Step 4: Compile and train the model

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
# Train the model for 10 epochs
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

## Step 5: Evaluate the model

Lets show the model loss and accuracy during training

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

# Calculate the accuracy for thre test dataset
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
# Print the final accuracy
print('Test Accuracy: ',test_acc)

## Step 6: Make predictions

In [ ]:
# Make prediction on test dataset
predictions = model.predict(X_test)
# Show predictions shape
print(' Prediction shape', predictions)

In [ ]:
# Calculate the predicted label for test dataset
y_preds= np.argmax(predictions, axis=-1)
print(y_preds.shape)

In [ ]:

score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

### Create the confussion matrix

In [ ]:
# Calculate the confussion matrix
cm = confusion_matrix(y_test, y_preds)
print('Confusion Matrix\n')
print(cm)

Plotting the confussion matrix will show efficiently how model works. We define a function to plot the matrix

In [ ]:
# Function to plot the confussion matrix
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.2f}; misclass={:0.2f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
# Plot confussion matrix
plot_confusion_matrix(cm, class_names)

## Step 8: Save the model

to save our model we make a dir and call the save method

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
# Save the model
model.save('saved_model/my_model')

## Restore the model saved

When the model is saved using the save method, we can restore and load the model in a new model just calling the load_model. This procedure

In [ ]:
# Load the saved model
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(X_test, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

In [ ]:
# Calculate the predicted label for test dataset
y_preds= np.argmax(new_model.predict(X_test), axis=-1)
# Calculate the confussion matrix
cm = confusion_matrix(y_test, y_preds)
# Plot confussion matrix
plot_confusion_matrix(cm, class_names)